In [1]:
from functools import partial

import tensorflow as tf
import tensorflow_federated as tff

from ocddetection.learning.federated.stateful import evaluation
from ocddetection.learning.federated.stateful.impl import interpolation

In [2]:
window_size = 150
batch_size = 128
hidden_size = 64

In [3]:
val = evaluation.__load_data(
    '/opportunity/augmented/including_original',
    window_size,
    batch_size
)

In [4]:
server_state, client_state_fn, model_fn = interpolation.create(
    window_size,
    hidden_size,
    evaluation.__optimizer_fn,
    evaluation.__metrics_fn
)

In [5]:
client_states = {i: client_state_fn(idx) for idx, i in enumerate(val.clients)}

In [9]:
ckpt_manager = tff.simulation.FileCheckpointManager('./checkpoints/federated/interpolation')

In [10]:
ckpt = ckpt_manager.load_latest_checkpoint([server_state, list(client_states.values())])[0]

In [16]:
server_state, client_states = ckpt

In [24]:
client_dict = dict(zip(val.clients, client_states))

In [28]:
[client_dict[client].mixing_coefficient for client in val.clients]

[<tf.Tensor: shape=(), dtype=float32, numpy=0.77100605>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.6658855>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.7021341>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.78530556>]

In [ ]:
cm = val.data[val.clients[0]].reduce(
    tf.zeros((2, 2), dtype=tf.int32),
    partial(evaluation.__evaluation_step, model=model)
)

In [ ]:
cm

In [ ]:
outputs = model.report_local_outputs()

In [ ]:
outputs['auc']

In [ ]:
metrics = evaluation.__metrics_fn()

In [ ]:
metrics[0].result()

In [ ]:
for variable, value in zip(metrics[0].variables, outputs['auc']):
    variable.assign(value)

In [ ]:
metrics[0].result()

In [ ]:
for variable, value in zip(metrics[1].variables, outputs['precision']):
    variable.assign(value)

In [ ]:
metrics[1].result()